In [1]:
import pandas as pd
import numpy as np

c:\Users\spradkyl\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


### Preparing the dataset 

Use only the following columns:

* `'engine_displacement'`,
* `'horsepower'`,
* `'vehicle_weight'`,
* `'model_year'`,
* `'fuel_efficiency_mpg'`


In [3]:
df_subset = df[['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']]
df_subset.head()

,engine_displacement,horsepower,vehicle_weight,model_year,fuel_efficiency_mpg
0,170,159.0,3413.433759,2003,13.231729
1,130,97.0,3149.664934,2007,13.688217
2,170,78.0,3079.038997,2018,14.246341
3,220,NaN,2542.392402,2009,16.912736
4,210,140.0,3460.870990,2009,12.488369


### Question 1

There's one column with missing values. What is it?

* `'engine_displacement'`
* `'horsepower'`
* `'vehicle_weight'`
* `'model_year'`

In [5]:
df_subset.isna().sum()#[df_subset.isna().sum() > 0]

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

### Question 2

What's the median (50% percentile) for variable `'horsepower'`?

- 49
- 99
- 149
- 199

In [8]:
df_subset['horsepower'].describe()

count    8996.000000
mean      149.657292
std        29.879555
min        37.000000
25%       130.000000
50%       149.000000
75%       170.000000
max       271.000000
Name: horsepower, dtype: float64

In [9]:
df_subset.horsepower.median()

149.0

### Prepare and split the dataset

* Shuffle the dataset (the filtered one you created above), use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.

Use the same code as in the lectures

In [16]:
n = len(df_subset)
n_val = int(n*.2)
n_test = int(n*.2)
n_train = n - (n_test+n_val)


n_val,n_test,n_train

(1940, 1940, 5824)

In [21]:
#shuffling the data
np.random.seed(42)
idx = np.arange(n)
np.random.shuffle(idx)

df_shuffle = df_subset.iloc[idx]

In [22]:
#splitting the data
df_train = df_shuffle.iloc[:n_train]
df_val = df_shuffle.iloc[n_train:n_train+n_val]
df_test = df_shuffle.iloc[n_train+n_val:]

len(df_train), len(df_val), len(df_test)

(5824, 1940, 1940)

In [23]:
# reset index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [24]:
# original values from the target
y_train_orig = df_train['fuel_efficiency_mpg']
y_val_orig = df_val['fuel_efficiency_mpg']
y_test_orig = df_test['fuel_efficiency_mpg']

y_train_orig

0       16.642943
1       16.298377
2       18.591822
3       11.818843
4       19.402209
          ...    
5819    16.964054
5820    10.059094
5821    17.593237
5822    18.925748
5823    17.965284
Name: fuel_efficiency_mpg, Length: 5824, dtype: float64

In [26]:
#applying log transformation to Fuel_Efficiency
y_train = np.log1p(df_train['fuel_efficiency_mpg'].values)
y_val = np.log1p(df_val['fuel_efficiency_mpg'].values)
y_test = np.log1p(df_test['fuel_efficiency_mpg'].values)

In [27]:
# To avoid accidentally using the target variable
del df_train['fuel_efficiency_mpg']
del df_val['fuel_efficiency_mpg']
del df_test['fuel_efficiency_mpg']

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

Options:

- With 0
- With mean
- Both are equally good

In [28]:
df_train.columns

Index(['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year'], dtype='object')

In [52]:
# prepare X

base_features = ['engine_displacement', 'horsepower', 'vehicle_weight', 'model_year']

def prepare_X(df: pd.DataFrame, fillna_value: float):
    df_num = df[base_features]
    df_num = df_num.fillna(fillna_value)

    x = df_num.values
    
    return x

In [56]:
def train_linear_regression(X: np.ndarray, y: np.ndarray):
    X_0 = np.ones(X.shape[0])
    X = np.column_stack([X_0,X])

    XTX = X.T.dot(X)
    XTX_inverse = np.linalg.inv(XTX)

    w = XTX_inverse.dot(X.T).dot(y)

    return w[0], w[1:]

In [62]:
def rmse(y,y_pred):
    se = (y-y_pred)**2
    mse = se.mean()
    return np.sqrt(mse)

In [57]:
mean = df_train['horsepower'].mean()

x_train_mean = prepare_X(df_train,mean)
w_0_mean, w_mean = train_linear_regression(x_train_mean, y_train)

In [61]:
x_val_mean = prepare_X(df_val,mean)
y_val_pred_mean = w_0_mean + x_val_mean.dot(w_mean)

In [63]:
round(rmse(y_val_pred_mean,y_val),2)

0.04

In [49]:
# Linear regression

def linear_regression(X, y):
    X_0 = np.ones(X.shape[0])
    X = np.column_stack([X_0,X])
    
    XTX = X.T.dot(X) # multiplying the transpose of X by X itself.
    XTX_inverse = np.linalg.inv(XTX)
    
    w = XTX_inverse.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [ ]:
# RMSE 

def rmse(y, y_pred):
    se = (y - y_pred)**2
    mse = se.mean()
    return np.sqrt(mse)

#### Linear Regression (Mean)

In [50]:
mean = df_train['horsepower'].mean()

X_mean_train = prepare_X(df_train, fillna_value=mean)
w_0_mean, w_mean = linear_regression(X_mean_train, y_train)

In [51]:
X_mean_val = prepare_X(df_val, fillna_value=mean)

y_mean_pred_val = w_0_mean + X_mean_val.dot(w_mean)
y_mean_pred_val

array([2.8068845 , 2.74624076, 2.56829206, ..., 2.69564277, 2.67673323,
       2.83627891])

In [44]:
round(rmse(y_val, y_mean_pred_val),2)

0.04

#### Linear Regression (Zero)

In [ ]:
X_null_train = prepare_X(df_train, fillna_value=0)
w_0_null, w_null = linear_regression(X_null_train, y_train)


(3.664666247205269,
 array([ 2.14827083e-06,  2.21209296e-04, -3.23460699e-04,  1.64673232e-05]))

In [48]:
X_null_val = prepare_X(df_val, fillna_value=0)

y_null_pred_val = w_0_null + X_null_val.dot(w_null)

y_null_pred_val

array([2.82641584, 2.75411816, 2.55686533, ..., 2.6745201 , 2.67029761,
       2.85296899])

In [51]:
round(rmse(y_val,y_null_pred_val),2)

0.04